In this code i want to show you how to use memory 
and checkpoint so that langgraph remember the conversation

In [1]:
from typing import Annotated
from dotenv import load_dotenv
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import ToolNode
from langgraph.graph import START, END
from langgraph.prebuilt import tools_condition
from langchain_community.tools import Tool
from langchain_community.utilities import GoogleSerperAPIWrapper

load_dotenv(override=True)



# Create the Serper search tool
search = GoogleSerperAPIWrapper()
serper_tool = Tool(
    name="Search",
    func=search.run,
    description="Useful for when you need to answer questions about current events or find up-to-date information"
)
tools = [serper_tool]

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

# chat bot with tools is the function which will use the llm with tool knowledge
# llm with tools know about the tools and decide if it wants to do a tool call or not
def chat_bot_with_tools(state: State):
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    llm_with_tools = llm.bind_tools(tools)
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

# tool node is the node which will be used to call the tools
toolNode = ToolNode(tools=tools)

graph_builder.add_node("chat_bot", chat_bot_with_tools)
graph_builder.add_node("tools", toolNode)

graph_builder.add_edge(START, "chat_bot")
graph_builder.add_conditional_edges("chat_bot", tools_condition, "tools")
graph_builder.add_edge("tools", "chat_bot")
graph_builder.add_edge("chat_bot", END)

Now the memory part

In [2]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

Now to use the memory we need a config

In [3]:
config = {"configurable": {"thread_id": "1"}}

Now let us chat

In [ ]:
import gradio as gr

def chat(message, history):
    # partial_response = ""
    # for event in graph.stream({"messages": [{"role": "user", "content": message}]}):
    #     for value in event.values():
    #         print("Assistant:", value["messages"][-1].content)
    #         partial_response += value["messages"][-1].content
    #         yield partial_response
    response = graph.invoke({"messages": [{"role": "user", "content": message}]}, config)
    return response["messages"][-1].content
    


chat_interface = gr.ChatInterface(chat, type="messages", theme="soft")
chat_interface.launch()

In [5]:
chat_interface.close()

Closing server running on port: 7860
